In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, RobustScaler, StandardScaler

In [3]:
df = pd.read_csv("~/ProjectData/NFL/all_data_9_19.csv", low_memory=False)

General plan for feature / target selector function:

1. Select position and seasons, sort by player / gid / set dcp and home to string
2. Take any int64 or float64 in df.ix[:,:34] and use a RobustScaler on it and reassign (on the whole subset)
3. Select by player in set(player) _*using ix_
4. Shift(-1) on target/ ou/ imp_score/ home/ temp/ surf/ cond/ stad/ pstat/ gstat/ details/ dpc 
5. Delete cols: unnamed / gid / player/ fname / lname / team / opponent / pos1 / seas
6. Take int64 and float64 in temp.ix[:,34:] and use an imputer to fill nas with mean of cols
7. Take any int64 or float64 col in temp.ix[:,34:] and use a RobustScaler on it and reassign
8. Recompile the temps into a new set.
9. Get dummies on any object type cols
10. Return the full set (x,y separation and some additional filtering will happen in a second function)

In [14]:
def make_feature(pos, year, df, week = None):

    """processes the df resulting from the NFL database query getting offensive player stats"""
    
    step1 = df.ix[(df.pos1 == pos) & (df.seas > year),:].sort_values(["player", "gid"]) #select pos/year from df
    step1.reset_index(drop=1, inplace=1)
    
    step1.dcp = step1.dcp.astype(int).astype(str)
    step1.home = step1.home.astype(int).astype(str)
    
    step1.ix[:,35:].dropna(axis = 1, how = "all", inplace =1) #drop all na cols, these would come after row 35
                                                              #but hardcoding just to make sure
    
    mask1 = step1.ix[:,35:].columns[(step1.ix[:,35:].dtypes == "int64")| (step1.ix[:,35:].dtypes == "float64")]
    
    #fill nans and standardize the average columns per player
    
    step1[mask1] = step1[mask1]/1.0 #convert to floats
    
    step2 = pd.DataFrame()
    shiftcols = ["target", "ou", "imp_score", "home", "temp", "surf",
                 "cond", "stad", "pstat", "gstat", "details", "dcp"] 
    
    ###SHIFTCOLS ARE IMPORTANT. THESE WOULD HAVE TO BE SOURCED TO MAKE FUTURE PREDICTIONS SINCE THEY ARE SHIFTED
    ###TO THE FUTURE ALONG WITH THE TARGET (include schedule/ injury / vegas / depth chart data)
    
    for player in set(step1.player): #loop through each player and fill in nas with mean values in stat cols
        
        temp = step1.ix[step1.player == player,:]
        temp.loc[:,shiftcols] = temp[shiftcols].shift(-1)
        Imputer(copy=0).fit_transform(temp[mask1])
        step2 = step2.append(temp)
        
    mask2 = step2.columns[(step2.dtypes == "float64")].difference(["target"])
    
    step2.loc[:,mask2]=step2[mask2].fillna(value = 0)
  
    step2.loc[:,mask2]=RobustScaler().fit_transform(step2[mask2])
    step2["wk_str"] = step2.wk.astype(str)
    
    step3 = step2.drop(["Unnamed: 0", "tname", "index", "gid.1",
                       "pos1"], axis = 1)
    stand_cols = ["ou", "imp_score", "height", "weight", "age_in_season", "forty", "bench", "vertical",\
                 "broad", "shuttle", "cone", "arm", "hand", "dpos", "yrs_exp"]
    
   
    step3.loc[:,stand_cols] = step3.loc[:,stand_cols] /1.0
    step3.loc[:,stand_cols]=Imputer().fit_transform(step3[stand_cols])
   
    step3.loc[:,stand_cols]=RobustScaler().fit_transform(step3[stand_cols])
    
    
    step3 = pd.get_dummies(step3, prefix = "dummy", columns =["team", "opponent", 
                                                              "pstat", "gstat", "details", "stad", "temp",
                                                             "surf", "cond", "dcp", "home", 
                                                             "wk_str"]  ,drop_first=1)
    
    
    return step3.reset_index(drop=1)

In [ ]:
def crop_and_separate(games_lim, top_fraction ,df = df, feat = 1):
    
    """returns X,y after being passed a nfl make_feature df"""
    result = df.dropna()
    player_list1 = result.player.value_counts()[result.player.value_counts() > games_lim].index 
    #get players with more than a certain number of games played
    pivot = result.pivot_table(values = ["target"], index = "player")
    player_list2 = pivot.sort_values("target", ascending=0)[:len(pivot)/top_fraction].index 
    #grab the top fraction of players by avg points(2 would grab top 50%, 5 top 20% etc)
    result = result[result.player.isin(player_list1) & result.player.isin(player_list2)]
    y = result.target
    X = result.ix[:,result.columns.get_loc("seas")+1:] #all features are to the right of 'seas' col
    if feat ==1:
        return X, y
    elif feat == "data":
        return X, y, result
    else: return X
    

In [18]:
testWR = make_feature("WR", 2000, df = df)
testWR.head()

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_2,dummy_20,dummy_21,dummy_3,dummy_4,dummy_5,dummy_6,dummy_7,dummy_8,dummy_9
19610,293,AW-1000,3,0.0,Alvis,Whitted,2001,-0.257329,0.616277,-0.24753,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8643,332,AW-1000,6,1.7,Alvis,Whitted,2001,-1.041326,-1.549001,-0.24753,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5842,346,AW-1000,7,0.4,Alvis,Whitted,2001,-0.498559,-0.682890,-0.24753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
83362,369,AW-1000,8,2.0,Alvis,Whitted,2001,-0.257329,0.183221,-0.24753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16982,380,AW-1000,9,0.0,Alvis,Whitted,2001,-0.800096,-1.356532,-0.24753,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
testRB = make_feature("RB", 2000, df = df)
testRB.head()

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_2,dummy_20,dummy_21,dummy_3,dummy_4,dummy_5,dummy_6,dummy_7,dummy_8,dummy_9
66472,323,MC-1600,5,0.0,Mike,Cloud,2001,0.066894,0.289987,-0.567215,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
209,333,MC-1600,6,0.0,Mike,Cloud,2001,0.605595,0.193845,-0.567215,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
35725,345,MC-1600,7,9.5,Mike,Cloud,2001,-0.112674,0.097703,-0.567215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
72165,410,MC-1600,11,1.6,Mike,Cloud,2001,0.366172,-0.575292,-0.567215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60929,441,MC-1600,13,1.5,Mike,Cloud,2001,-0.651375,-0.719505,-0.567215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
testQB = make_feature("QB", 2000, df = df)
testQB.head()

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_2,dummy_20,dummy_21,dummy_3,dummy_4,dummy_5,dummy_6,dummy_7,dummy_8,dummy_9
75296,774,SC-1500,17,NaN,Scott,Covington,2002,-5.030481,-4.003478,-0.712141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64365,1345,AS-1600,2,1.36,Alex,Smith,2005,0.238001,-0.109412,0.480598,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
886,1378,AS-1600,4,-5.74,Alex,Smith,2005,0.601345,-0.926685,0.480598,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
36466,1390,AS-1600,5,0.44,Alex,Smith,2005,-0.609800,-1.743958,0.480598,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
86779,1420,AS-1600,7,2.60,Alex,Smith,2005,0.298559,-0.061338,0.480598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
testWR.columns[testWR.isnull().sum() >0]

Index([u'target'], dtype='object')